In [1]:
# Importacion de librerias

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración estética
# sns.set_theme(style="whitegrid")

In [2]:
# Carga de datos

url_demo = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_demo.txt'
url_web1 = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_1.txt'
url_web2 = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_web_data_pt_2.txt'
url_experiment = 'https://raw.githubusercontent.com/data-bootcamp-v4/lessons/refs/heads/main/5_6_eda_inf_stats_tableau/project/files_for_project/df_final_experiment_clients.txt'

df_demo = pd.read_csv(url_demo)
df_web_1 = pd.read_csv(url_web1)
df_web_2 = pd.read_csv(url_web2)
df_experiment = pd.read_csv(url_experiment)

# 🧩 Unimos pt_1 y pt_2 del web data
df_web = pd.concat([df_web_1, df_web_2], ignore_index=True)

In [4]:
# Vista general de los datasets

In [3]:
# 🔍 Primer vistazo a los datos
print("Demo:")
display(df_demo.head())

print("\nWeb:")
display(df_web.head())

print("\nExperiment:")
display(df_experiment.head())

Demo:


,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,836976,6.0,73.0,60.5,U,2.0,45105.30,6.0,9.0
1,2304905,7.0,94.0,58.0,U,2.0,110860.30,6.0,9.0
2,1439522,5.0,64.0,32.0,U,2.0,52467.79,6.0,9.0
3,1562045,16.0,198.0,49.0,M,2.0,67454.65,3.0,6.0
4,5126305,12.0,145.0,33.0,F,2.0,103671.75,0.0,3.0



Web:


,client_id,visitor_id,visit_id,process_step,date_time
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04



Experiment:


,client_id,Variation
0,9988021,Test
1,8320017,Test
2,4033851,Control
3,1982004,Test
4,9294070,Control


In [4]:
# Agrupamos por client_id y contamos cuántos visit_id distintos tiene cada uno
visitas_por_cliente = df_web.groupby('client_id')['visit_id'].nunique()

# Contamos cuántos clientes tienen más de una visita
num_clientes_multivisita = (visitas_por_cliente > 1).sum()

print(f"Número de clientes con más de una visita distinta: {num_clientes_multivisita}")
print(f"Total de clientes únicos: {visitas_por_cliente.shape[0]}")

Número de clientes con más de una visita distinta: 27861
Total de clientes únicos: 120157


In [5]:
# Merge total de los datasets

# 🔗 Unimos todos los datasets por 'client_id'
df = df_web.merge(df_experiment, on='client_id', how='left')
df = df.merge(df_demo, on='client_id', how='left')

print(f"Shape final del dataset combinado: {df.shape}")
df.head()

Shape final del dataset combinado: (755405, 14)


,client_id,visitor_id,visit_id,process_step,date_time,Variation,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth
0,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:27:07,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
1,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:26:51,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
2,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:19:22,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
3,9988021,580560515_7732621733,781255054_21935453173_531117,step_2,2017-04-17 15:19:13,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0
4,9988021,580560515_7732621733,781255054_21935453173_531117,step_3,2017-04-17 15:18:04,Test,5.0,64.0,79.0,U,2.0,189023.86,1.0,4.0


In [6]:
df.columns

Index(['client_id', 'visitor_id', 'visit_id', 'process_step', 'date_time',
       'Variation', 'clnt_tenure_yr', 'clnt_tenure_mnth', 'clnt_age', 'gendr',
       'num_accts', 'bal', 'calls_6_mnth', 'logons_6_mnth'],
      dtype='object')

In [8]:
# Renombrado de columnas

# Normalizar nombres de columnas: minúsculas y sin abreviaturas confusas
df.rename(columns={
    'client_id': 'client_id',
    'visitor_id': 'visitor_id',
    'visit_id': 'visit_id',
    'process_step': 'process_step',
    'date_time': 'date_time',
    'Variation': 'variation',
    'clnt_tenure_yr': 'tenure_years',
    'clnt_tenure_mnth': 'tenure_months',
    'clnt_age': 'age',
    'gendr': 'gender',
    'num_accts': 'num_accounts',
    'bal': 'balance',
    'calls_6_mnth': 'calls_6_months',
    'logons_6_mnth': 'logons_6_months'
}, inplace=True)

In [9]:
cols_to_drop = [
    "visitor_id",
    "calls_6_months",
    "logons_6_months",
    "tenure_months",
    "num_accounts",
    "gender",
    "balance"
]

df.drop(columns=cols_to_drop, inplace=True)

In [10]:
# Tipos de datos, nulos y duplicados
print("Tipos de datos:")
print(df.dtypes)

print("\nValores nulos:")
print(df.isna().sum())

print("\nDuplicados:")
print(df.duplicated().sum())

Tipos de datos:
client_id         int64
visit_id         object
process_step     object
date_time        object
variation        object
tenure_years    float64
age             float64
dtype: object

Valores nulos:
client_id            0
visit_id             0
process_step         0
date_time            0
variation       434096
tenure_years    305689
age             305701
dtype: int64

Duplicados:
10764


In [11]:
# Convertimos formato de la columna date_time a date_time

df['date_time'] = pd.to_datetime(df['date_time'])

In [12]:
# Comprobamos si hay algun client_id que tenga más de un valor distinto en columnas clave

for col in ['variation', 'age']:
    inconsistencias = df.groupby('client_id')[col].nunique()
    print(f"{col}: valores inconsistentes → {(inconsistencias > 1).sum()} clientes")

variation: valores inconsistentes → 0 clientes
age: valores inconsistentes → 0 clientes


In [13]:
# Inspección total de valores únicos con sus frecuencias
for col in df.columns:
    print(f"\n📌 {col}")
    print(df[col].value_counts(dropna=False))
    print("-" * 60)


📌 client_id
client_id
5574879    111
2274658     90
2128341     84
5951379     81
1595223     81
          ... 
1200258      1
4532140      1
1418419      1
1740430      1
1279022      1
Name: count, Length: 120157, dtype: int64
------------------------------------------------------------

📌 visit_id
visit_id
875138661_34710212496_881092    104
518285126_8572733997_709761      88
602953935_48759866176_238903     80
961878360_85895454962_607105     80
308874104_3998249411_902973      72
                               ... 
593062957_98089749948_583560      1
483131122_43428812832_224802      1
185961647_84781157409_178026      1
701927685_66645026413_783707      1
148166449_67114410166_524360      1
Name: count, Length: 158095, dtype: int64
------------------------------------------------------------

📌 process_step
process_step
start      243945
step_1     163193
step_2     133062
step_3     112242
confirm    102963
Name: count, dtype: int64
--------------------------------------------

In [14]:
# Filtro 1: solo clientes con datos del experimento

df = df[df['variation'].notna()]

In [15]:
# Filtro 2: clientes con datos por edad:

df = df[df['age'].notna()]


In [16]:
# Filtro 3: clientes con datos sobre la fecha (hora/minuto/segundo) por step:

df = df[df['date_time'].notna()]

In [17]:
df.shape

(321195, 7)

In [18]:
df['tenure_years'].value_counts(dropna=False)

tenure_years
6.0     31829
5.0     31339
7.0     22742
4.0     20915
15.0    20568
8.0     18072
14.0    16572
13.0    16242
12.0    15852
9.0     14975
10.0    12997
11.0    12786
16.0    12129
21.0    10519
20.0     7377
22.0     6801
17.0     6221
23.0     6183
18.0     5720
19.0     5482
24.0     4218
27.0     3285
26.0     3168
28.0     2335
25.0     2243
3.0      2214
30.0     1595
29.0     1412
33.0     1076
32.0      880
34.0      794
31.0      780
35.0      541
2.0       410
36.0      204
37.0      183
38.0      129
39.0       62
48.0       60
40.0       40
46.0       33
55.0       30
50.0       28
47.0       23
43.0       23
51.0       20
42.0       19
44.0       17
49.0       16
52.0       11
54.0       10
41.0        9
45.0        6
Name: count, dtype: int64

In [19]:
# Contar duplicados exactos en client_id, process_step y date_time
duplicados = df.duplicated(subset=['client_id', 'process_step', 'date_time'], keep=False)
num_duplicados = duplicados.sum()

print(f"Número de duplicados exactos (client_id + process_step + date_time): {num_duplicados}")

Número de duplicados exactos (client_id + process_step + date_time): 7119


In [20]:
# Filtrar el DataFrame original para ver solo los duplicados exactos
duplicados_exactos = df[df.duplicated(subset=['client_id', 'process_step', 'date_time'], keep=False)]
duplicados_exactos.head(20)

,client_id,visit_id,process_step,date_time,variation,tenure_years,age
364,9358626,223297395_36250329195_832161,start,2017-04-28 12:27:28,Control,5.0,24.5
365,9358626,223297395_36250329195_832161,start,2017-04-28 12:27:28,Control,5.0,24.5
367,5376894,688984457_43441834354_912755,start,2017-04-28 14:20:01,Test,13.0,48.0
368,5376894,688984457_43441834354_912755,start,2017-04-28 14:20:01,Test,13.0,48.0
382,438567,330543236_29863358529_771432,start,2017-04-28 02:52:29,Test,15.0,38.0
383,438567,330543236_29863358529_771432,start,2017-04-28 02:52:29,Test,15.0,38.0
784,1423857,568819166_80596727019_75889,confirm,2017-04-10 22:44:58,Control,8.0,37.5
785,1423857,568819166_80596727019_75889,confirm,2017-04-10 22:44:58,Control,8.0,37.5
1815,372523,25331354_74479847014_854987,start,2017-04-27 10:51:26,Control,6.0,57.5
1816,372523,25331354_74479847014_854987,start,2017-04-27 10:51:26,Control,6.0,57.5


In [21]:
# Guardado de duplicados exactos en CSV para revisar fuera
duplicados_exactos.to_csv('../data/duplicados_exactos.csv', index=False)

In [22]:
# Eliminamos duplicados exactos (conservamos solo una aparición)
df = df.drop_duplicates(subset=['client_id', 'process_step', 'date_time'])

In [23]:
df.shape

(317123, 7)

In [24]:
# Creamos columna variable binaria: completed = True si llegó a 'confirm'

df['completed'] = df['process_step'] == 'confirm'
print(df['completed'].value_counts(normalize=True))


completed
False    0.864649
True     0.135351
Name: proportion, dtype: float64


In [25]:
# Creamos columna total de pasos (incluyendo repeticiones)

df['step_count'] = 1  # se sumará luego por cliente

In [26]:
# Creamos columna tiempo entre pasos (ordenado previamente)

df = df.sort_values(by=['client_id', 'date_time']).copy()
df['time_diff'] = df.groupby('client_id')['date_time'].diff().dt.total_seconds()

In [27]:
# Exportación

df_export = df.groupby('client_id').agg({
    'variation': 'first',
    'completed': 'max',
    'process_step': ['nunique', 'count'],
    'date_time': ['min', 'max'],
    'time_diff': 'mean',
    'age': 'first',
    'tenure_years': 'first'
})

# Aplanar columnas multinivel
df_export.columns = ['_'.join(col).strip() for col in df_export.columns.values] # Después del .agg() con múltiples funciones por columna (['nunique', 'count'], etc.), los nombres de las columnas son tuplas multinivel, como: ('process_step', 'nunique'), ('date_time', 'max'). Este código convierte esos nombres en strings planos: 'process_step_nunique', 'date_time_max'

# Duración total del proceso
df_export['duration_total_secs'] = (
    df_export['date_time_max'] - df_export['date_time_min']
).dt.total_seconds()

# Exportación final
df_export.to_csv('../data/processed/df_clean_export.csv', index=True)

In [28]:
df_export.head()

,variation_first,completed_max,process_step_nunique,process_step_count,date_time_min,date_time_max,time_diff_mean,age_first,tenure_years_first,duration_total_secs
client_id,,,,,,,,,,
555,Test,True,5,5,2017-04-15 12:57:56,2017-04-15 13:00:34,39.500000,29.5,3.0,158.0
647,Test,True,5,5,2017-04-12 15:41:28,2017-04-12 15:47:45,94.250000,57.5,12.0,377.0
934,Test,False,1,4,2017-04-18 02:36:30,2017-04-18 02:38:52,47.333333,51.0,9.0,142.0
1028,Control,False,4,9,2017-04-08 18:51:28,2017-04-08 19:00:26,67.250000,36.0,12.0,538.0
1104,Control,False,1,2,2017-06-12 07:49:18,2017-06-20 22:31:33,744135.000000,48.0,5.0,744135.0


In [29]:
df_export.shape

(50487, 10)

In [30]:
# Exportación versión no agregada para futuros análisis (incluyendo visit_id)
df.to_csv('../data/processed/df_clean_full.csv', index=False)

In [31]:
# 🔁 Agrupar por visita individual
df_visits = df.sort_values(by=["client_id", "visit_id", "date_time"]).copy()
df_visits["step_count"] = 1
df_visits["time_diff"] = df_visits.groupby("visit_id")["date_time"].diff().dt.total_seconds()

# Agrupación por visita
df_visits_agg = df_visits.groupby("visit_id").agg({
    "client_id": "first",
    "variation": "first",
    "process_step": ["nunique", "count"],
    "completed": "max",
    "date_time": ["min", "max"],
    "time_diff": "mean",
    "age": "first",
    "tenure_years": "first"
})

# Aplanado y renombrado
df_visits_agg.columns = ['_'.join(col).strip() for col in df_visits_agg.columns.values]
df_visits_agg.rename(columns={
    "process_step_nunique": "steps_unique",
    "process_step_count": "steps_total",
    "completed_max": "completed",
    "date_time_min": "start_time",
    "date_time_max": "end_time",
    "time_diff_mean": "avg_time_between_steps"
}, inplace=True)

# Calcular duración
df_visits_agg["duration_total_secs"] = (
    df_visits_agg["end_time"] - df_visits_agg["start_time"]
).dt.total_seconds()

# Reset index y exportar
df_visits_agg.reset_index(inplace=True)
df_visits_agg.to_csv("../data/processed/df_clean_visits.csv", index=False)
